**Import des données**

In [2]:
import pandas as pd

# Charger et concaténer
years = ['2020', '2021', '2022', '2023', '2024', '2025']
df_vf = pd.concat([
    pd.read_csv(f"../../data/DVF_{year}.csv", sep=",", low_memory=False)
    for year in years
], ignore_index=True)




In [3]:
print(df_vf.columns)
print(df_vf.dtypes)

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')
id_mutation                      object
date_mutation                    object
numero_disposition                int64
natu

In [ ]:

# Filtrer Île-de-France
departements_idf = ['75', '77', '78', '91', '92', '93', '94', '95']
df_vf_idf = df_vf[df_vf['code_departement'].isin(departements_idf)].copy()

# Supprimer colonnes inutiles (adaptées aux nouvelles colonnes)
colonnes_a_supprimer = [
    'numero_disposition', 'adresse_suffixe', 'adresse_code_voie',
    'ancien_code_commune', 'ancien_nom_commune', 'ancien_id_parcelle',
    'numero_volume', 'code_nature_culture', 'code_nature_culture_speciale'
]
# Supprimer uniquement les colonnes qui existent
colonnes_a_supprimer = [col for col in colonnes_a_supprimer if col in df_vf_idf.columns]
df_vf_idf.drop(columns=colonnes_a_supprimer, inplace=True)

# Convertir date et créer variables temporelles
df_vf_idf['date_mutation'] = pd.to_datetime(df_vf_idf['date_mutation'], format='%Y-%m-%d', errors='coerce')
df_vf_idf['annee'] = df_vf_idf['date_mutation'].dt.year
df_vf_idf['mois'] = df_vf_idf['date_mutation'].dt.month
df_vf_idf['trimestre'] = df_vf_idf['date_mutation'].dt.quarter
df_vf_idf['annee_trimestre'] = df_vf_idf['annee'].astype(str) + '-T' + df_vf_idf['trimestre'].astype(str)

# Conversions numériques
numeric_cols = ['valeur_fonciere', 'surface_reelle_bati', 'surface_terrain',
                'nombre_pieces_principales', 'nombre_lots']
for col in numeric_cols:
    if col in df_vf_idf.columns:
        if df_vf_idf[col].dtype == 'object':
            df_vf_idf[col] = df_vf_idf[col].str.replace(',', '.').astype(float)
        else:
            df_vf_idf[col] = pd.to_numeric(df_vf_idf[col], errors='coerce')

# Filtres qualité
df_vf_idf_filtered = df_vf_idf[
    (df_vf_idf['valeur_fonciere'] > 1000) &
    (df_vf_idf['surface_reelle_bati'] > 9) &
    (df_vf_idf['surface_reelle_bati'] < 1000) &
    (df_vf_idf['nombre_pieces_principales'] >= 1) &
    (df_vf_idf['nombre_pieces_principales'] <= 15)
].copy()

# Prix au m²
df_vf_idf_filtered['prix_m2'] = df_vf_idf_filtered['valeur_fonciere'] / df_vf_idf_filtered['surface_reelle_bati']

# Filtrer prix au m² aberrants
df_vf_idf_filtered = df_vf_idf_filtered[
    (df_vf_idf_filtered['prix_m2'] > 500) &
    (df_vf_idf_filtered['prix_m2'] < 20000)
].copy()

# GÉOCODAGE : Créer identifiant de quartier/secteur
df_vf_idf_filtered['quartier'] = (
    df_vf_idf_filtered['nom_commune'].fillna('') +
    ' (' + df_vf_idf_filtered['code_postal'].astype(str).str.zfill(5) + ')'
)

# Pour Paris, utiliser l'arrondissement
df_vf_idf_filtered['arrondissement'] = df_vf_idf_filtered['code_postal'].astype(str).str[-2:].where(
    df_vf_idf_filtered['code_departement'] == '75',
    other=None
)
df_vf_idf_filtered['quartier_detaille'] = df_vf_idf_filtered.apply(
    lambda x: f"Paris {x['arrondissement']}e" if pd.notna(x['arrondissement']) else x['quartier'],
    axis=1
)

# PRIX MÉDIAN PAR QUARTIER ET TRIMESTRE
prix_median_quartier_trimestre = df_vf_idf_filtered.groupby(['quartier_detaille', 'annee_trimestre'])['prix_m2'].agg([
    ('prix_median_m2', 'median'),
    ('nb_transactions', 'count'),
    ('prix_moyen_m2', 'mean')
]).reset_index()

# Filtrer les quartiers avec au moins 5 transactions
prix_median_quartier_trimestre = prix_median_quartier_trimestre[
    prix_median_quartier_trimestre['nb_transactions'] >= 5
]

# Joindre le prix médian au dataframe principal
df_vf_idf_filtered = df_vf_idf_filtered.merge(
    prix_median_quartier_trimestre[['quartier_detaille', 'annee_trimestre', 'prix_median_m2']],
    on=['quartier_detaille', 'annee_trimestre'],
    how='left'
)

# Calculer l'écart par rapport au prix médian
df_vf_idf_filtered['ecart_prix_median_pct'] = (
    (df_vf_idf_filtered['prix_m2'] - df_vf_idf_filtered['prix_median_m2']) /
    df_vf_idf_filtered['prix_median_m2'] * 100
)

# Trier et réinitialiser l'index
df_vf_idf_filtered.sort_values('date_mutation', inplace=True)
df_vf_idf_filtered.reset_index(drop=True, inplace=True)

print(f"Dataset nettoyé : {df_vf_idf_filtered.shape}")
print(f"\nPrix médians calculés pour {prix_median_quartier_trimestre.shape[0]} combinaisons quartier-trimestre")
print("\nAperçu des colonnes créées :")
print(df_vf_idf_filtered[['date_mutation', 'quartier_detaille', 'annee_trimestre', 'prix_m2', 'prix_median_m2', 'ecart_prix_median_pct','latitude','longitude']].head(10))

df_vf_idf_filtered.to_csv("../../data/df_vf_idf.csv", index=False)



Dataset nettoyé : (815674, 41)

Prix médians calculés pour 14838 combinaisons quartier-trimestre

Aperçu des colonnes créées :
  date_mutation                 quartier_detaille annee_trimestre  \
0    2020-07-01            Saint-Mammès (77670.0)         2020-T3   
1    2020-07-01         L'Haÿ-les-Roses (94240.0)         2020-T3   
2    2020-07-01         Vitry-sur-Seine (94400.0)         2020-T3   
3    2020-07-01              Versailles (78000.0)         2020-T3   
4    2020-07-01  Montigny-le-Bretonneux (78180.0)         2020-T3   
5    2020-07-01           Choisy-le-Roi (94600.0)         2020-T3   
6    2020-07-01     Vélizy-Villacoublay (78140.0)         2020-T3   
7    2020-07-01         Vitry-sur-Seine (94400.0)         2020-T3   
8    2020-07-01          Ivry-sur-Seine (94200.0)         2020-T3   
9    2020-07-01                Viroflay (78220.0)         2020-T3   

        prix_m2  prix_median_m2  ecart_prix_median_pct   latitude  longitude  
0   2348.571429     2502.013809   

Détection des outliers / valeurs manquantes

In [3]:
import numpy as np
df_vf_idf_filtered = df_vf_idf_filtered.replace(['',' '], np.nan)
valeurs_manquantes_par_colonne = df_vf_idf_filtered.isna().sum()
nombre_lignes = df_vf_idf_filtered['id_mutation'].count()

for colonne, nombre_valeurs_manquantes in valeurs_manquantes_par_colonne.items():
    if nombre_valeurs_manquantes != 0:  
        print(f"Colonne '{colonne}': {round(nombre_valeurs_manquantes/nombre_lignes,2)} %")


Colonne 'adresse_numero': 0.02 %
Colonne 'adresse_nom_voie': 0.0 %
Colonne 'code_postal': 0.0 %
Colonne 'lot1_numero': 0.31 %
Colonne 'lot1_surface_carrez': 0.61 %
Colonne 'lot2_numero': 0.67 %
Colonne 'lot2_surface_carrez': 0.9 %
Colonne 'lot3_numero': 0.96 %
Colonne 'lot3_surface_carrez': 0.99 %
Colonne 'lot4_numero': 0.99 %
Colonne 'lot4_surface_carrez': 1.0 %
Colonne 'lot5_numero': 1.0 %
Colonne 'lot5_surface_carrez': 1.0 %
Colonne 'nature_culture': 0.69 %
Colonne 'nature_culture_speciale': 0.97 %
Colonne 'surface_terrain': 0.69 %
Colonne 'longitude': 0.01 %
Colonne 'latitude': 0.01 %
Colonne 'arrondissement': 0.82 %
Colonne 'prix_median_m2': 0.02 %
Colonne 'ecart_prix_median_pct': 0.02 %


Pour les colonnes, pour lesquelles on a des valeurs manquantes, le taux est au maximum de 1%

In [4]:
print(df_vf_idf_filtered.dtypes)

id_mutation                          object
date_mutation                datetime64[ns]
nature_mutation                      object
valeur_fonciere                     float64
adresse_numero                      float64
adresse_nom_voie                     object
code_postal                         float64
code_commune                         object
nom_commune                          object
code_departement                     object
id_parcelle                          object
lot1_numero                          object
lot1_surface_carrez                 float64
lot2_numero                          object
lot2_surface_carrez                 float64
lot3_numero                          object
lot3_surface_carrez                 float64
lot4_numero                          object
lot4_surface_carrez                 float64
lot5_numero                          object
lot5_surface_carrez                 float64
nombre_lots                           int64
code_type_local                 

In [ ]:
import pandas as pd
df_evolution_ipc = pd.read_csv("../../data/Indice_Prix_Conso/valeurs_mensuelles.csv", sep=";")

In [59]:
print(df_evolution_ipc.describe())

       Libellé  \
count       85   
unique      85   
top     idBank   
freq         1   

       Indice des prix à la consommation - Base 2015 - Ensemble des ménages - France - Alimentation  \
count                                                  84                                             
unique                                                 80                                             
top                                                131.52                                             
freq                                                    3                                             

       Codes  
count     82  
unique     2  
top        A  
freq      81  


In [60]:
print(df_evolution_ipc.dtypes)


Libellé                                                                                         object
Indice des prix à la consommation - Base 2015 - Ensemble des ménages - France - Alimentation    object
Codes                                                                                           object
dtype: object


In [61]:
print(df_evolution_ipc.tail(10))

    Libellé  \
75  2019-10   
76  2019-09   
77  2019-08   
78  2019-07   
79  2019-06   
80  2019-05   
81  2019-04   
82  2019-03   
83  2019-02   
84  2019-01   

   Indice des prix à la consommation - Base 2015 - Ensemble des ménages - France - Alimentation  \
75                                             106.15                                             
76                                             106.62                                             
77                                             107.15                                             
78                                             106.66                                             
79                                              106.1                                             
80                                             105.99                                             
81                                             105.27                                             
82                                        

In [62]:
import pandas as pd


df_evolution_ipc = df_evolution_ipc.rename(columns={
    df_evolution_ipc.columns[0]: "Date_maj",
    df_evolution_ipc.columns[1]: "IPC_base_2015"
})

df_evolution_ipc = df_evolution_ipc.drop([0, 1, 2]).reset_index(drop=True)

df_evolution_ipc["Date_maj"] = pd.to_datetime(df_evolution_ipc["Date_maj"], format="%Y-%m", errors="coerce")

df_evolution_ipc["Date_maj"] = df_evolution_ipc["Date_maj"] + pd.offsets.MonthEnd(0)

df_evolution_ipc = df_evolution_ipc[df_evolution_ipc["Date_maj"] < "2025-07"]

df_evolution_ipc = df_evolution_ipc.drop(columns=[df_evolution_ipc.columns[-1]])

print(df_evolution_ipc.head(10))



     Date_maj IPC_base_2015
4  2025-06-30        133.59
5  2025-05-31        133.75
6  2025-04-30        133.14
7  2025-03-31        132.19
8  2025-02-28        131.84
9  2025-01-31        131.94
10 2024-12-31        131.54
11 2024-11-30        131.72
12 2024-10-31        131.78
13 2024-09-30        131.52


In [63]:
print(df_evolution_ipc.tail())

     Date_maj IPC_base_2015
77 2019-05-31        105.99
78 2019-04-30        105.27
79 2019-03-31         105.2
80 2019-02-28        105.41
81 2019-01-31        105.23


In [64]:
df_vf_idf_filtered_sorted = df_vf_idf.sort_values(by="date_mutation", ascending=False)
print(df_vf_idf_filtered_sorted["date_mutation"].head(10))
print(df_vf_idf_filtered_sorted["date_mutation"].tail(10))


20046618   2025-06-30
20028709   2025-06-30
20085567   2025-06-30
20028712   2025-06-30
20028713   2025-06-30
20028714   2025-06-30
20028715   2025-06-30
20028716   2025-06-30
20028717   2025-06-30
20028718   2025-06-30
Name: date_mutation, dtype: datetime64[ns]
1908087   2020-07-01
1629851   2020-07-01
1629852   2020-07-01
1908090   2020-07-01
1908089   2020-07-01
1908088   2020-07-01
1629853   2020-07-01
1629854   2020-07-01
1629855   2020-07-01
1569723   2020-07-01
Name: date_mutation, dtype: datetime64[ns]


Dans notre dataset, on s'arrête au 2025 - 06 - 30

Ouverture et récupération de l'indice de confiance des ménages

In [ ]:
import openpyxl
import pandas as pd
df_indice_confiance_menage = pd.read_excel("../../data/indice_confiance_menage_synth/245_IR_Camme.xlsx")



In [24]:
print(df_indice_confiance_menage.columns)

Index(['Opinion des ménages-Monthly confidence consumer survey', 'Unnamed: 1',
       'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12'],
      dtype='object')


In [26]:
# Garder uniquement les colonnes 0 et 1
df_indice_confiance_menage = df_indice_confiance_menage.iloc[:, [0, 1]]
df_indice_confiance_menage = df_indice_confiance_menage.drop([0,1,2,3,4,5,6]).reset_index(drop=True)

In [27]:
print(df_indice_confiance_menage.columns)

Index(['Opinion des ménages-Monthly confidence consumer survey', 'Unnamed: 1'], dtype='object')


In [28]:
df_indice_confiance_menage = df_indice_confiance_menage.rename(columns={
    df_indice_confiance_menage.columns[0]: "Date_maj",
    df_indice_confiance_menage.columns[1]: "Indice_confiance_menage"
})



In [29]:
print(df_indice_confiance_menage)

                Date_maj Indice_confiance_menage
0    1973-05-01 00:00:00              131.094251
1    1973-10-01 00:00:00              125.091056
2    1974-01-01 00:00:00              112.615392
3    1974-05-01 00:00:00               117.12321
4    1974-10-01 00:00:00              108.117889
..                   ...                     ...
501  2025-05-01 00:00:00               88.158058
502  2025-06-01 00:00:00               88.244595
503  2025-07-01 00:00:00               88.310865
504  2025-08-01 00:00:00               86.991779
505  2025-09-01 00:00:00               87.380365

[506 rows x 2 columns]


In [30]:
df_indice_confiance_menage["Date_maj"] = pd.to_datetime(df_indice_confiance_menage["Date_maj"], format="%Y-%m", errors="coerce")
print(df_indice_confiance_menage)

      Date_maj Indice_confiance_menage
0   1973-05-01              131.094251
1   1973-10-01              125.091056
2   1974-01-01              112.615392
3   1974-05-01               117.12321
4   1974-10-01              108.117889
..         ...                     ...
501 2025-05-01               88.158058
502 2025-06-01               88.244595
503 2025-07-01               88.310865
504 2025-08-01               86.991779
505 2025-09-01               87.380365

[506 rows x 2 columns]


In [37]:
df_indice_confiance_menage["Date_maj"] = df_indice_confiance_menage["Date_maj"] + pd.offsets.MonthEnd(0)
print(df_indice_confiance_menage)

      Date_maj Indice_confiance_menage
505 2025-09-30               87.380365
504 2025-08-31               86.991779
503 2025-07-31               88.310865
502 2025-06-30               88.244595
501 2025-05-31               88.158058
..         ...                     ...
429 2019-05-31              100.296249
428 2019-04-30               99.176006
427 2019-03-31               98.058276
426 2019-02-28               96.910125
425 2019-01-31               94.446904

[81 rows x 2 columns]


In [38]:
print(df_indice_confiance_menage)

      Date_maj Indice_confiance_menage
505 2025-09-30               87.380365
504 2025-08-31               86.991779
503 2025-07-31               88.310865
502 2025-06-30               88.244595
501 2025-05-31               88.158058
..         ...                     ...
429 2019-05-31              100.296249
428 2019-04-30               99.176006
427 2019-03-31               98.058276
426 2019-02-28               96.910125
425 2019-01-31               94.446904

[81 rows x 2 columns]


In [39]:
df_indice_confiance_menage = df_indice_confiance_menage.sort_values(by="Date_maj", ascending=False)

In [40]:
print(df_indice_confiance_menage)

      Date_maj Indice_confiance_menage
505 2025-09-30               87.380365
504 2025-08-31               86.991779
503 2025-07-31               88.310865
502 2025-06-30               88.244595
501 2025-05-31               88.158058
..         ...                     ...
429 2019-05-31              100.296249
428 2019-04-30               99.176006
427 2019-03-31               98.058276
426 2019-02-28               96.910125
425 2019-01-31               94.446904

[81 rows x 2 columns]


In [41]:
# 1. Convertir la colonne "Date_maj" en datetime (si ce n'est pas déjà fait)
df_indice_confiance_menage["Date_maj"] = pd.to_datetime(df_indice_confiance_menage["Date_maj"], errors="coerce")

# 2. Filtrer les dates entre janvier 2020 et septembre 2025
df_indice_confiance_menage = df_indice_confiance_menage[
    (df_indice_confiance_menage["Date_maj"] >= "2019-01-01") &
    (df_indice_confiance_menage["Date_maj"] < "2025-10-01")  
]

# Afficher le résultat
print(df_indice_confiance_menage.head(10))


      Date_maj Indice_confiance_menage
505 2025-09-30               87.380365
504 2025-08-31               86.991779
503 2025-07-31               88.310865
502 2025-06-30               88.244595
501 2025-05-31               88.158058
500 2025-04-30               91.212198
499 2025-03-31               91.188831
498 2025-02-28               93.237874
497 2025-01-31               91.550424
496 2024-12-31               88.371446


In [42]:
print(df_indice_confiance_menage.tail(10))

      Date_maj Indice_confiance_menage
434 2019-10-31              104.877148
433 2019-09-30              104.504536
432 2019-08-31               103.21054
431 2019-07-31              102.598424
430 2019-06-30              101.586633
429 2019-05-31              100.296249
428 2019-04-30               99.176006
427 2019-03-31               98.058276
426 2019-02-28               96.910125
425 2019-01-31               94.446904


Evolution du PIB en %, base 2020

In [ ]:
df_evolution_pib_volume_base2020 = pd.read_excel("../../data/PIB/econ-gen-pib-composante-trim.xlsx")

In [44]:
print(df_evolution_pib_volume_base2020.columns)

Index(['Évolution du produit intérieur brut et de ses composantes',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')


In [45]:
df_evolution_pib_volume_base2020 = df_evolution_pib_volume_base2020.rename(columns={
    df_evolution_pib_volume_base2020.columns[0]: "Date",
    df_evolution_pib_volume_base2020.columns[1]: "Evolution_PIB_volume_base2020"
})

In [46]:
df_evolution_pib_volume_base2020 = df_evolution_pib_volume_base2020.iloc[:, [0, 1]]

In [25]:
print(df_evolution_pib_volume_base2020)

                                                  Date  \
0                                                  NaN   
1    par rapport au trimestre précédent en volume en %   
2                                            Trimestre   
3                                                  NaN   
4                                              2025-T3   
..                                                 ...   
310                      1. Administrations publiques.   
311  Note : données révisées ; les volumes sont mes...   
312  Lecture : au 3e trimestre 2025, le produit int...   
313                                    Champ : France.   
314  Source : Insee, comptes nationaux trimestriels...   

    Evolution_PIB_volume_base2020  
0                             NaN  
1                             NaN  
2    Produit intérieur brut (PIB)  
3                             NaN  
4                        0.504328  
..                            ...  
310                           NaN  
311            

In [47]:
df_evolution_pib_volume_base2020 = df_evolution_pib_volume_base2020.drop([0,1,2,3]).reset_index(drop=True)

In [64]:
print(df_evolution_pib_volume_base2020)

                                                  Date  \
0                                              2025-T3   
1                                              2025-T2   
2                                              2025-T1   
3                                              2024-T4   
4                                              2024-T3   
..                                                 ...   
306                      1. Administrations publiques.   
307  Note : données révisées ; les volumes sont mes...   
308  Lecture : au 3e trimestre 2025, le produit int...   
309                                    Champ : France.   
310  Source : Insee, comptes nationaux trimestriels...   

    Evolution_PIB_volume_base2020  
0                        0.504328  
1                        0.333917  
2                        0.088369  
3                       -0.047944  
4                        0.339255  
..                            ...  
306                           NaN  
307            

In [48]:
import pandas as pd
from datetime import datetime
from typing import Union, Optional
def trimestre_to_date_fin(trim_str: str) -> Optional[datetime]:
    """
    Convertit un string de format 'AAAA-TX' en date de FIN du trimestre.
    
    Args:
        trim_str (str): String au format 'AAAA-TX' (ex: '2020-T1')
    
    Returns:
        datetime or None: Date du dernier jour du trimestre ou None si conversion impossible
    """
    try:
        # Nettoyer la string (enlever les espaces, etc.)
        trim_str = str(trim_str).strip()
        
        # Vérifier le format basique
        if '-' not in trim_str or len(trim_str) < 6:
            return None
            
        # Extraire année et trimestre
        annee_part = trim_str.split('-')[0]  # '2020'
        trimestre_part = trim_str.split('-')[1]  # 'T1'
        
        # Garder seulement les chiffres
        annee = ''.join(filter(str.isdigit, annee_part))
        trimestre = ''.join(filter(str.isdigit, trimestre_part))
        
        if not annee or not trimestre:
            return None
            
        annee = int(annee)
        trimestre = int(trimestre)
        
        # Validation
        if not (1 <= trimestre <= 4):
            return None
            
        # Date de FIN du trimestre
        if trimestre == 1:    # T1 → 31 mars
            return datetime(annee, 3, 31)
        elif trimestre == 2:  # T2 → 30 juin
            return datetime(annee, 6, 30)
        elif trimestre == 3:  # T3 → 30 septembre
            return datetime(annee, 9, 30)
        else:                 # T4 → 31 décembre
            return datetime(annee, 12, 31)
        
    except (ValueError, IndexError, TypeError):
        return None

# Application avec vérification
df_evolution_pib_volume_base2020['Date_Modif'] = df_evolution_pib_volume_base2020['Date'].apply(trimestre_to_date_fin)

# Vérifier les valeurs qui n'ont pas pu être converties
valeurs_problematiques = df_evolution_pib_volume_base2020[df_evolution_pib_volume_base2020['Date_Modif'].isna()]['Date'].unique()
print("Valeurs non converties:", valeurs_problematiques)

# Aperçu des résultats
print("\nAperçu des conversions:")
print(df_evolution_pib_volume_base2020[['Date', 'Date_Modif']].head(10))

Valeurs non converties: ['1. Administrations publiques.'
 "Note : données révisées\xa0; les volumes sont mesurés aux prix de l'année précédente chaînés et corrigés des variations saisonnières et des effets des jours ouvrables."
 'Lecture\xa0: au 3e trimestre 2025, le produit intérieur brut (PIB) en volume augmente de 0,5 % par rapport au trimestre précédent.'
 'Champ\xa0: France.'
 'Source : Insee, comptes nationaux trimestriels - base 2020.']

Aperçu des conversions:
      Date Date_Modif
0  2025-T3 2025-09-30
1  2025-T2 2025-06-30
2  2025-T1 2025-03-31
3  2024-T4 2024-12-31
4  2024-T3 2024-09-30
5  2024-T2 2024-06-30
6  2024-T1 2024-03-31
7  2023-T4 2023-12-31
8  2023-T3 2023-09-30
9  2023-T2 2023-06-30


In [49]:
df_evolution_pib_volume_base2020 = df_evolution_pib_volume_base2020[df_evolution_pib_volume_base2020["Date_Modif"]>="2019-01-01"]

In [77]:
print(df_evolution_pib_volume_base2020.head())

      Date Evolution_PIB_volume_base2020 Date_Modif
0  2025-T3                      0.504328 2025-09-30
1  2025-T2                      0.333917 2025-06-30
2  2025-T1                      0.088369 2025-03-31
3  2024-T4                     -0.047944 2024-12-31
4  2024-T3                      0.339255 2024-09-30


In [72]:
print(df_evolution_pib_volume_base2020.dtypes)

Date                                     object
Evolution_PIB_volume_base2020            object
Date_Modif                       datetime64[ns]
dtype: object


Importation des données relatives aux taux moyen des crédits du secteur concurrentiel (hors assurance et coût des sûretés), source : Observatoire Crédit Logement CSA

In [ ]:
df_taux_credit_moyen = pd.read_excel("../../data/taux_interet_immo_moyen_particulier/Taux_moyen_credit_bancaire_immobilier_particulier_trimestriel.xlsx")

In [51]:
print(df_taux_credit_moyen)

       Date  \
0   2019-T1   
1   2019-T2   
2   2019-T3   
3   2019-T4   
4   2020-T1   
5   2020-T2   
6   2020-T3   
7   2020-T4   
8   2021-T1   
9   2021-T2   
10  2021-T3   
11  2021-T4   
12  2022-T1   
13  2022-T2   
14  2022-T3   
15  2022-T4   
16  2023-T1   
17  2023-T2   
18  2023-T3   
19  2023-T4   
20  2024-T1   
21  2024-T2   
22  2024-T3   
23  2024-T4   
24  2025-T1   
25  2025-T2   

    Taux des prêts du secteur concurrentiel (hors assurance et coût des sureteé) en moyenne  
0                                              0.0142                                        
1                                              0.0129                                        
2                                              0.0119                                        
3                                              0.0113                                        
4                                              0.0113                                        
5                             

In [52]:
df_taux_credit_moyen = df_taux_credit_moyen.rename(columns={
    df_taux_credit_moyen.columns[0]: "Date_Trimestriel",
    df_taux_credit_moyen.columns[1]: "Taux_moyen_credit_immo"
})

In [53]:
print(df_taux_credit_moyen)

   Date_Trimestriel  Taux_moyen_credit_immo
0           2019-T1                  0.0142
1           2019-T2                  0.0129
2           2019-T3                  0.0119
3           2019-T4                  0.0113
4           2020-T1                  0.0113
5           2020-T2                  0.0129
6           2020-T3                  0.0123
7           2020-T4                  0.0120
8           2021-T1                  0.0113
9           2021-T2                  0.0106
10          2021-T3                  0.0105
11          2021-T4                  0.0105
12          2022-T1                  0.0112
13          2022-T2                  0.0140
14          2022-T3                  0.0178
15          2022-T4                  0.0222
16          2023-T1                  0.0284
17          2023-T2                  0.0329
18          2023-T3                  0.0377
19          2023-T4                  0.0420
20          2024-T1                  0.0399
21          2024-T2             

In [54]:
df_taux_credit_moyen['Date_Modif'] = df_taux_credit_moyen['Date_Trimestriel'].apply(trimestre_to_date_fin)

In [70]:
print(df_taux_credit_moyen)

   Date_Trimestriel  Taux_moyen_credit_immo Date_Modif
0           2019-T1                  0.0142 2019-03-31
1           2019-T2                  0.0129 2019-06-30
2           2019-T3                  0.0119 2019-09-30
3           2019-T4                  0.0113 2019-12-31
4           2020-T1                  0.0113 2020-03-31
5           2020-T2                  0.0129 2020-06-30
6           2020-T3                  0.0123 2020-09-30
7           2020-T4                  0.0120 2020-12-31
8           2021-T1                  0.0113 2021-03-31
9           2021-T2                  0.0106 2021-06-30
10          2021-T3                  0.0105 2021-09-30
11          2021-T4                  0.0105 2021-12-31
12          2022-T1                  0.0112 2022-03-31
13          2022-T2                  0.0140 2022-06-30
14          2022-T3                  0.0178 2022-09-30
15          2022-T4                  0.0222 2022-12-31
16          2023-T1                  0.0284 2023-03-31
17        

In [56]:
print(df_taux_credit_moyen.columns)
print(df_evolution_pib_volume_base2020.columns)
print(df_indice_confiance_menage.columns)
print(df_evolution_ipc.columns)


Index(['Date_Trimestriel', 'Taux_moyen_credit_immo', 'Date_Modif'], dtype='object')
Index(['Date', 'Evolution_PIB_volume_base2020', 'Date_Modif'], dtype='object')
Index(['Date_maj', 'Indice_confiance_menage'], dtype='object')
Index(['Date_maj', 'IPC_base_2015'], dtype='object')


In [109]:
print(df_vf_idf.columns)

Index(['id_mutation', 'date_mutation', 'nature_mutation', 'valeur_fonciere',
       'adresse_numero', 'adresse_nom_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'id_parcelle', 'lot1_numero',
       'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez',
       'lot3_numero', 'lot3_surface_carrez', 'lot4_numero',
       'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez',
       'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati',
       'nombre_pieces_principales', 'nature_culture',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude',
       'annee', 'mois', 'trimestre', 'annee_trimestre'],
      dtype='object')


In [112]:
print(df_vf_idf["date_mutation"].dtypes)

datetime64[ns]


In [57]:
print(df_vf_idf["date_mutation"].head)

<bound method NDFrame.head of 1569723    2020-07-01
1569724    2020-07-01
1569725    2020-07-01
1569726    2020-07-01
1569727    2020-07-01
              ...    
20102734   2025-06-27
20102735   2025-06-27
20102736   2025-06-27
20102737   2025-06-25
20102738   2025-06-25
Name: date_mutation, Length: 2393734, dtype: datetime64[ns]>


In [115]:
print(df_taux_credit_moyen["Date_Modif"].dtypes)
print(df_evolution_pib_volume_base2020["Date_Modif"].dtypes)
print(df_indice_confiance_menage["Date_maj"].dtypes)
print(df_evolution_ipc["Date_maj"].dtypes)

datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]


Jointure des tables

In [ ]:
df_vf_idf_enriched = pd.read_csv("../../data/df_vf_idf_enriched.csv")
print(df_vf_idf_enriched.head(10))

/tmp/ipykernel_1804/1316083096.py:1: DtypeWarning: Columns (11,13,15,17,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vf_idf_enriched = pd.read_csv("../data/df_vf_idf_enriched.csv")


   id_mutation date_mutation nature_mutation  valeur_fonciere  adresse_numero  \
0  2020-621368    2020-07-01           Vente         246600.0            40.0   
1  2020-621368    2020-07-01           Vente         246600.0             NaN   
2  2020-621368    2020-07-01           Vente         246600.0             NaN   
3  2020-621368    2020-07-01           Vente         246600.0             NaN   
4  2020-621369    2020-07-01           Vente         424000.0             5.0   
5  2020-621369    2020-07-01           Vente         424000.0             5.0   
6  2020-621370    2020-07-01           Vente         192000.0           140.0   
7  2020-621371    2020-07-02           Vente         160000.0            54.0   
8  2020-621372    2020-07-02           Vente         175000.0          5086.0   
9  2020-621372    2020-07-02           Vente         175000.0          5086.0   

        adresse_nom_voie  code_postal  code_commune      nom_commune  \
0  SEN DES LONGUES RAIES      77670.

In [ ]:
import pandas as pd
import numpy as np
from typing import List
import gc
import os

# ========================================
# CRÉATION DU DOSSIER SI NÉCESSAIRE
# ========================================

os.makedirs('../data', exist_ok=True)
print("Dossier '../data' prêt\n")

# ========================================
# FONCTIONS DE BASE (IDENTIQUES)
# ========================================

def creer_lags_mensuels(
    df_base: pd.DataFrame, 
    df_macro: pd.DataFrame, 
    col_date_base: str, 
    col_date_macro: str, 
    colonnes_features: List[str], 
    lags: List[int] = [1, 2, 3, 6]
) -> pd.DataFrame:
    """Ajoute des lags mensuels pour éviter le look-ahead bias"""
    df_result = df_base.copy()
    
    # Préparer les données macro
    df_macro = df_macro.copy()
    df_macro[col_date_macro] = pd.to_datetime(df_macro[col_date_macro], errors='coerce')
    
    # Convertir en numérique et float32
    for col in colonnes_features:
        if col in df_macro.columns:
            df_macro[col] = pd.to_numeric(df_macro[col], errors='coerce').astype('float32')
    
    # Pour chaque lag
    for lag in lags:
        print(f"      → Création lag{lag}m...", end=" ", flush=True)
        
        date_ref = pd.to_datetime(df_result[col_date_base]) - pd.DateOffset(months=lag)
        date_ref = date_ref + pd.offsets.MonthEnd(0)
        
        df_temp = pd.merge(
            df_result[[col_date_base]],
            df_macro[[col_date_macro] + colonnes_features],
            left_on=date_ref,
            right_on=col_date_macro,
            how='left'
        )
        
        for col in colonnes_features:
            if col in df_temp.columns:
                df_result[f'{col}_lag{lag}m'] = df_temp[col].values
        
        del df_temp, date_ref
        gc.collect()
        print("ok")
    
    return df_result


def creer_lags_trimestriels(
    df_base: pd.DataFrame, 
    df_macro: pd.DataFrame, 
    col_date_base: str, 
    col_date_macro: str, 
    colonnes_features: List[str], 
    lags: List[int] = [1, 2, 3, 4]
) -> pd.DataFrame:
    """Ajoute des lags trimestriels pour éviter le look-ahead bias"""
    df_result = df_base.copy()
    
    # Préparer les données macro
    df_macro = df_macro.copy()
    df_macro[col_date_macro] = pd.to_datetime(df_macro[col_date_macro], errors='coerce')
    
    # Convertir en numérique et float32
    for col in colonnes_features:
        if col in df_macro.columns:
            df_macro[col] = pd.to_numeric(df_macro[col], errors='coerce').astype('float32')
    
    # Pour chaque lag
    for lag in lags:
        print(f"      → Création lag{lag}t...", end=" ", flush=True)
        
        date_ref = pd.to_datetime(df_result[col_date_base]) - pd.DateOffset(months=lag*3)
        date_ref = date_ref + pd.offsets.QuarterEnd(0)
        
        df_temp = pd.merge(
            df_result[[col_date_base]],
            df_macro[[col_date_macro] + colonnes_features],
            left_on=date_ref,
            right_on=col_date_macro,
            how='left'
        )
        
        for col in colonnes_features:
            if col in df_temp.columns:
                df_result[f'{col}_lag{lag}t'] = df_temp[col].values
        
        del df_temp, date_ref
        gc.collect()
        print("ok")
    
    return df_result


# ========================================
# ÉTAPE 1 : TAUX CRÉDIT IMMOBILIER
# ========================================

print("=" * 70)
print("ÉTAPE 1/4 : TAUX CRÉDIT IMMOBILIER (TRIMESTRIEL)")
print("=" * 70)

# Charger le dataset de base
df_step1 = df_vf_idf.copy()
df_step1['date_mutation'] = pd.to_datetime(df_step1['date_mutation'])

print(f"\nDataset initial : {len(df_step1):,} transactions")
print(f"   Colonnes initiales : {len(df_step1.columns)}")

# Préparer données taux
df_taux = df_taux_credit_moyen.copy()
df_taux['Date_Modif'] = pd.to_datetime(df_taux['Date_Modif'], errors='coerce') 

print(f"\nPériode macro : {df_taux['Date_Modif'].min().date()} à {df_taux['Date_Modif'].max().date()}")
print(f"Nombre de points : {len(df_taux)}")
print(f"Aperçu des dates :")
print(df_taux[['Date_Modif', 'Taux_moyen_credit_immo']].head(10))

# Intégrer les lags
df_step1 = creer_lags_trimestriels(
    df_base=df_step1,
    df_macro=df_taux,
    col_date_base='date_mutation',
    col_date_macro='Date_Modif',
    colonnes_features=['Taux_moyen_credit_immo'],
    lags=[1, 2, 3, 4]
)

nb_features = len([c for c in df_step1.columns if 'Taux_moyen_credit_immo' in c])
print(f"\nFeatures créées : {nb_features}")
print(f"   Nouvelles colonnes : {len(df_step1.columns) - len(df_vf_idf.columns)}")

# Vérifier si on a des données
nb_non_null = df_step1['Taux_moyen_credit_immo_lag1t'].notna().sum()
print(f"   Valeurs non-nulles pour lag1t : {nb_non_null:,} ({nb_non_null/len(df_step1)*100:.1f}%)")

# Sauvegarder
print("\nSauvegarde intermédiaire...")
df_step1.to_csv('../../data/df_step1_taux_credit.csv', index=False)
print("Sauvegardé : ../../data/df_step1_taux_credit.csv")

# Libérer mémoire
del df_taux
gc.collect()

print("\n" + "=" * 70)
input("Appuie sur ENTRÉE pour continuer avec l'étape 2 (PIB)...")


# ========================================
# ÉTAPE 2 : ÉVOLUTION PIB
# ========================================

print("\n" + "=" * 70)
print("ÉTAPE 2/4 : ÉVOLUTION PIB (TRIMESTRIEL)")
print("=" * 70)

# Charger le résultat de l'étape précédente
df_step2 = df_step1.copy()
print(f"\nDataset après étape 1 : {len(df_step2):,} transactions, {len(df_step2.columns)} colonnes")

# Préparer données PIB
df_pib = df_evolution_pib_volume_base2020.copy()
df_pib['Date_Modif'] = pd.to_datetime(df_pib['Date_Modif'], errors='coerce')  

print(f"\nPériode macro : {df_pib['Date_Modif'].min().date()} à {df_pib['Date_Modif'].max().date()}")
print(f"Nombre de points : {len(df_pib)}")
print(f"Aperçu des dates :")
print(df_pib[['Date_Modif', 'Evolution_PIB_volume_base2020']].head(10))

# Intégrer les lags
df_step2 = creer_lags_trimestriels(
    df_base=df_step2,
    df_macro=df_pib,
    col_date_base='date_mutation',
    col_date_macro='Date_Modif', 
    colonnes_features=['Evolution_PIB_volume_base2020'],
    lags=[1, 2, 3, 4]
)

nb_features = len([c for c in df_step2.columns if 'Evolution_PIB' in c])
print(f"\nFeatures créées : {nb_features}")
print(f"   Colonnes totales : {len(df_step2.columns)}")

# Vérifier si on a des données
nb_non_null = df_step2['Evolution_PIB_volume_base2020_lag1t'].notna().sum()
print(f"   ✓ Valeurs non-nulles pour lag1t : {nb_non_null:,} ({nb_non_null/len(df_step2)*100:.1f}%)")

# Sauvegarder
print("\nSauvegarde intermédiaire...")
df_step2.to_csv('../../data/df_step2_pib.csv', index=False)
print("Sauvegardé : ../../data/df_step2_pib.csv")

# Libérer mémoire
del df_step1, df_pib
gc.collect()

print("\n" + "=" * 70)
input("Appuie sur ENTRÉE pour continuer avec l'étape 3 (Confiance)...")


# ========================================
# ÉTAPE 3 : INDICE CONFIANCE MÉNAGE
# ========================================

print("\n" + "=" * 70)
print("ÉTAPE 3/4 : INDICE CONFIANCE MÉNAGE (MENSUEL)")
print("=" * 70)

# Charger le résultat de l'étape précédente
df_step3 = df_step2.copy()
print(f"\n Dataset après étape 2 : {len(df_step3):,} transactions, {len(df_step3.columns)} colonnes")

# Préparer données confiance
df_confiance = df_indice_confiance_menage.copy()
df_confiance['Date_maj'] = pd.to_datetime(df_confiance['Date_maj'], errors='coerce')

print(f"\n Période macro : {df_confiance['Date_maj'].min().date()} à {df_confiance['Date_maj'].max().date()}")
print(f" Nombre de points : {len(df_confiance)}")

# Intégrer les lags
df_step3 = creer_lags_mensuels(
    df_base=df_step3,
    df_macro=df_confiance,
    col_date_base='date_mutation',
    col_date_macro='Date_maj',
    colonnes_features=['Indice_confiance_menage'],
    lags=[1, 2, 3, 6]
)

nb_features = len([c for c in df_step3.columns if 'Indice_confiance' in c])
print(f"\n Features créées : {nb_features}")
print(f"   Colonnes totales : {len(df_step3.columns)}")

# Sauvegarder
print("\n Sauvegarde intermédiaire...")
df_step3.to_csv('../../data/df_step3_confiance.csv', index=False)
print(" Sauvegardé : ../../data/df_step3_confiance.csv")

# Libérer mémoire
del df_step2, df_confiance
gc.collect()

print("\n" + "=" * 70)
input(" Appuie sur ENTRÉE pour continuer avec l'étape 4 (IPC/Inflation)...")


# ========================================
# ÉTAPE 4 : IPC INFLATION
# ========================================

print("\n" + "=" * 70)
print("ÉTAPE 4/4 : IPC INFLATION (MENSUEL)")
print("=" * 70)

# Charger le résultat de l'étape précédente
df_step4 = df_step3.copy()
print(f"\n Dataset après étape 3 : {len(df_step4):,} transactions, {len(df_step4.columns)} colonnes")

# Préparer données IPC
df_ipc = df_evolution_ipc.copy()
df_ipc['Date_maj'] = pd.to_datetime(df_ipc['Date_maj'], errors='coerce')

print(f"\n Période macro : {df_ipc['Date_maj'].min().date()} à {df_ipc['Date_maj'].max().date()}")
print(f" Nombre de points : {len(df_ipc)}")

# Intégrer les lags (réduit pour éviter crash)
df_step4 = creer_lags_mensuels(
    df_base=df_step4,
    df_macro=df_ipc,
    col_date_base='date_mutation',
    col_date_macro='Date_maj',
    colonnes_features=['IPC_base_2015'],
    lags=[1, 3, 12]
)

nb_features = len([c for c in df_step4.columns if 'IPC' in c])
print(f"\n Features créées : {nb_features}")
print(f"   Colonnes totales : {len(df_step4.columns)}")

# Sauvegarder résultat FINAL
print("\n Sauvegarde FINALE...")
df_step4.to_csv('../../data/df_vf_idf_enriched.csv', index=False)
print(" Sauvegardé : ../../data/df_vf_idf_enriched.csv")

# Libérer mémoire
del df_step3, df_ipc
gc.collect()


# ========================================
# RÉSUMÉ FINAL
# ========================================

print("\n" + "=" * 70)
print(" RÉSUMÉ FINAL")
print("=" * 70)

df_final = df_step4

# Liste des colonnes macro créées (UNIQUEMENT LES LAGS)
colonnes_macro = [
    col for col in df_final.columns 
    if any(keyword in col for keyword in [
        'Taux_moyen', 'Evolution_PIB', 'Indice_confiance', 'IPC'
    ]) and 'lag' in col
]

print(f"\n Dataset final : {len(df_final):,} lignes × {len(df_final.columns)} colonnes")
print(f"   Features macro créées (lags uniquement) : {len(colonnes_macro)}")

print(f"\n📋 Liste complète des features macro (lags) :")
for i, col in enumerate(sorted(colonnes_macro), 1):
    print(f"   {i:2d}. {col}")

# Statistiques sur les valeurs manquantes
print(f"\n  Valeurs manquantes :")
missing_stats = df_final[colonnes_macro].isnull().sum().sort_values(ascending=False)
missing_pct = (missing_stats / len(df_final) * 100).round(2)

for col in missing_stats.head(15).index:
    count = missing_stats[col]
    pct = missing_pct[col]
    status = "ok" if pct < 100 else  "ERREUR DANS LE MERGE PRESENCE DE VALEURS MANQUANTES"
    print(f"   {status} {col[:50]:50s} : {count:6,} ({pct:5.2f}%)")

# Vérifier la couverture temporelle
print(f"\n Première date avec données disponibles :")
features_lag1 = [
    'Taux_moyen_credit_immo_lag1t',
    'Evolution_PIB_volume_base2020_lag1t',
    'Indice_confiance_menage_lag1m',
    'IPC_base_2015_lag1m'
]

for feature in features_lag1:
    if feature in df_final.columns:
        first_valid = df_final[df_final[feature].notna()]['date_mutation'].min()
        if pd.notna(first_valid):
            print(f"   • {feature:45s} : {first_valid.date()}")
        else:
            print(f"   • {feature:45s} : AUCUNE DONNÉE")

print("\n" + "=" * 70)
print(" INTÉGRATION TERMINÉE !")
print("=" * 70)
print("\n Fichiers créés dans '../data' :")
print("   1. df_step1_taux_credit.csv")
print("   2. df_step2_pib.csv")
print("   3. df_step3_confiance.csv")
print("   4. df_vf_idf_enriched.csv  ← FICHIER FINAL")

📁 Dossier '../data' prêt

ÉTAPE 1/4 : TAUX CRÉDIT IMMOBILIER (TRIMESTRIEL)

📊 Dataset initial : 2,393,734 transactions
   Colonnes initiales : 35

📅 Période macro : 2019-03-31 à 2025-06-30
📈 Nombre de points : 26
🔍 Aperçu des dates :
  Date_Modif  Taux_moyen_credit_immo
0 2019-03-31                  0.0142
1 2019-06-30                  0.0129
2 2019-09-30                  0.0119
3 2019-12-31                  0.0113
4 2020-03-31                  0.0113
5 2020-06-30                  0.0129
6 2020-09-30                  0.0123
7 2020-12-31                  0.0120
8 2021-03-31                  0.0113
9 2021-06-30                  0.0106
      → Création lag1t... ✓
      → Création lag2t... ✓
      → Création lag3t... ✓
      → Création lag4t... ✓

✅ Features créées : 4
   Nouvelles colonnes : 4
   ✓ Valeurs non-nulles pour lag1t : 2,393,734 (100.0%)

💾 Sauvegarde intermédiaire...
✅ Sauvegardé : ../data/df_step1_taux_credit.csv


ÉTAPE 2/4 : ÉVOLUTION PIB (TRIMESTRIEL)

📊 Dataset après étap

In [ ]:
df_vf_idf_enriched = pd.read_csv("../../data/df_vf_idf_enriched.csv")
print(df_vf_idf_enriched.head(10))

/tmp/ipykernel_1809/1316083096.py:1: DtypeWarning: Columns (11,13,15,17,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vf_idf_enriched = pd.read_csv("../data/df_vf_idf_enriched.csv")


   id_mutation date_mutation nature_mutation  valeur_fonciere  adresse_numero  \
0  2020-621368    2020-07-01           Vente         246600.0            40.0   
1  2020-621368    2020-07-01           Vente         246600.0             NaN   
2  2020-621368    2020-07-01           Vente         246600.0             NaN   
3  2020-621368    2020-07-01           Vente         246600.0             NaN   
4  2020-621369    2020-07-01           Vente         424000.0             5.0   
5  2020-621369    2020-07-01           Vente         424000.0             5.0   
6  2020-621370    2020-07-01           Vente         192000.0           140.0   
7  2020-621371    2020-07-02           Vente         160000.0            54.0   
8  2020-621372    2020-07-02           Vente         175000.0          5086.0   
9  2020-621372    2020-07-02           Vente         175000.0          5086.0   

        adresse_nom_voie  code_postal  code_commune      nom_commune  \
0  SEN DES LONGUES RAIES      77670.

In [ ]:
df_vf_idf_enriched.tail(1)

         id_mutation date_mutation nature_mutation  valeur_fonciere  \
2393733  2025-502882    2025-06-25           Vente       24417580.0   

         adresse_numero adresse_nom_voie  code_postal  code_commune  \
2393733             NaN              NaN          NaN         75115   

                      nom_commune  code_departement  ...  \
2393733  Paris 15e Arrondissement                75  ...   

        Evolution_PIB_volume_base2020_lag2t  \
2393733                           -0.047944   

        Evolution_PIB_volume_base2020_lag3t  \
2393733                            0.339255   

         Evolution_PIB_volume_base2020_lag4t Indice_confiance_menage_lag1m  \
2393733                               0.1913                      88.15806   

         Indice_confiance_menage_lag2m Indice_confiance_menage_lag3m  \
2393733                        91.2122                     91.188835   

         Indice_confiance_menage_lag6m IPC_base_2015_lag1m  \
2393733                      88.371445 

In [69]:
import pandas as pd
import numpy as np

# ========================================
# DIAGNOSTIC : POURQUOI LES LAGS TRIMESTRIELS NE MARCHENT PAS ?
# ========================================

print("=" * 70)
print("🔍 DIAGNOSTIC DES DONNÉES TRIMESTRIELLES")
print("=" * 70)

# ========================================
# 1. EXAMINER LES DATES DANS LES DATAFRAMES MACRO
# ========================================

print("\n📅 ÉTAPE 1 : Examiner les dates des séries macro")
print("─" * 70)

# Taux crédit
df_taux = df_taux_credit_moyen.copy()
df_taux['Date_Trimestriel'] = pd.to_datetime(df_taux['Date_Trimestriel'], errors='coerce')

print("\n🔹 TAUX CRÉDIT (df_taux_credit_moyen)")
print(f"   Nombre de lignes : {len(df_taux)}")
print(f"   Type de Date_Trimestriel : {df_taux['Date_Trimestriel'].dtype}")
print(f"\n   Premières dates :")
print(df_taux[['Date_Trimestriel', 'Taux_moyen_credit_immo']].head(10))

print(f"\n   Dernières dates :")
print(df_taux[['Date_Trimestriel', 'Taux_moyen_credit_immo']].tail(10))

# Vérifier si les dates sont bien en fin de trimestre
df_taux['est_fin_trimestre'] = df_taux['Date_Trimestriel'] == (df_taux['Date_Trimestriel'] + pd.offsets.QuarterEnd(0))
print(f"\n   ✅ Dates en fin de trimestre : {df_taux['est_fin_trimestre'].sum()} / {len(df_taux)}")
print(f"   ❌ Dates PAS en fin de trimestre : {(~df_taux['est_fin_trimestre']).sum()}")

if not df_taux['est_fin_trimestre'].all():
    print("\n   ⚠️  Exemples de dates qui ne sont PAS en fin de trimestre :")
    print(df_taux[~df_taux['est_fin_trimestre']][['Date_Trimestriel', 'Taux_moyen_credit_immo']].head())

# PIB
df_pib = df_evolution_pib_volume_base2020.copy()
df_pib['Date'] = pd.to_datetime(df_pib['Date'], errors='coerce')

print("\n🔹 PIB (df_evolution_pib_volume_base2020)")
print(f"   Nombre de lignes : {len(df_pib)}")
print(f"   Type de Date : {df_pib['Date'].dtype}")
print(f"\n   Premières dates :")
print(df_pib[['Date', 'Evolution_PIB_volume_base2020']].head(10))

print(f"\n   Dernières dates :")
print(df_pib[['Date', 'Evolution_PIB_volume_base2020']].tail(10))

# Vérifier si les dates sont bien en fin de trimestre
df_pib['est_fin_trimestre'] = df_pib['Date'] == (df_pib['Date'] + pd.offsets.QuarterEnd(0))
print(f"\n   ✅ Dates en fin de trimestre : {df_pib['est_fin_trimestre'].sum()} / {len(df_pib)}")
print(f"   ❌ Dates PAS en fin de trimestre : {(~df_pib['est_fin_trimestre']).sum()}")

if not df_pib['est_fin_trimestre'].all():
    print("\n   ⚠️  Exemples de dates qui ne sont PAS en fin de trimestre :")
    print(df_pib[~df_pib['est_fin_trimestre']][['Date', 'Evolution_PIB_volume_base2020']].head())

# ========================================
# 2. EXAMINER LES DATES CALCULÉES DANS DVF
# ========================================

print("\n" + "=" * 70)
print("📅 ÉTAPE 2 : Examiner les dates calculées dans DVF")
print("─" * 70)

# Prendre un échantillon de transactions
df_sample = df_vf_idf.head(100).copy()
df_sample['date_mutation'] = pd.to_datetime(df_sample['date_mutation'])

# Calculer ce que serait la date de référence pour lag1t
df_sample['date_ref_lag1t'] = df_sample['date_mutation'] - pd.DateOffset(months=3)
df_sample['date_ref_lag1t_arrondie'] = df_sample['date_ref_lag1t'] + pd.offsets.QuarterEnd(0)

print("\n🔹 Échantillon de dates DVF et dates de référence calculées (lag1t)")
print(df_sample[['date_mutation', 'date_ref_lag1t', 'date_ref_lag1t_arrondie']].head(20))

# ========================================
# 3. TESTER LE MERGE MANUELLEMENT
# ========================================

print("\n" + "=" * 70)
print("🧪 ÉTAPE 3 : Tester le merge manuellement")
print("─" * 70)

# Test avec les 10 premières lignes
df_test = df_vf_idf.head(10).copy()
df_test['date_mutation'] = pd.to_datetime(df_test['date_mutation'])

# Calculer date de référence
df_test['date_ref_lag1t'] = df_test['date_mutation'] - pd.DateOffset(months=3)
df_test['date_ref_lag1t_arrondie'] = df_test['date_ref_lag1t'] + pd.offsets.QuarterEnd(0)

print("\n🔹 Dates DVF :")
print(df_test[['date_mutation', 'date_ref_lag1t_arrondie']].to_string())

print("\n🔹 Dates disponibles dans df_taux_credit_moyen :")
print(df_taux['Date_Trimestriel'].sort_values().unique())

# Tenter le merge
df_merge_test = pd.merge(
    df_test[['date_mutation', 'date_ref_lag1t_arrondie']],
    df_taux[['Date_Trimestriel', 'Taux_moyen_credit_immo']],
    left_on='date_ref_lag1t_arrondie',
    right_on='Date_Trimestriel',
    how='left'
)

print("\n🔹 Résultat du merge test :")
print(df_merge_test)

nb_match = df_merge_test['Taux_moyen_credit_immo'].notna().sum()
print(f"\n   Matches trouvés : {nb_match} / {len(df_test)}")

if nb_match == 0:
    print("\n   ❌ AUCUN MATCH ! Le problème est confirmé.")
    
    # Chercher les dates les plus proches
    print("\n   🔍 Analyse : cherchons les dates les plus proches...")
    
    for idx, row in df_test.iterrows():
        date_cherchee = row['date_ref_lag1t_arrondie']
        
        # Trouver la date la plus proche dans df_taux
        df_taux['diff'] = (df_taux['Date_Trimestriel'] - date_cherchee).abs()
        plus_proche = df_taux.loc[df_taux['diff'].idxmin()]
        
        print(f"\n   Transaction du {row['date_mutation'].date()}")
        print(f"      → Cherche : {date_cherchee.date()} ({date_cherchee})")
        print(f"      → Plus proche dans macro : {plus_proche['Date_Trimestriel'].date()} ({plus_proche['Date_Trimestriel']})")
        print(f"      → Différence : {plus_proche['diff']}")

# ========================================
# 4. IDENTIFIER LE PROBLÈME
# ========================================

print("\n" + "=" * 70)
print("💡 DIAGNOSTIC")
print("=" * 70)

# Vérifier si le problème vient du format des dates
date_dvf_example = df_test['date_ref_lag1t_arrondie'].iloc[0]
date_macro_example = df_taux['Date_Trimestriel'].iloc[0]

print(f"\n🔍 Comparaison détaillée des dates :")
print(f"\n   Date DVF (après calcul lag) :")
print(f"      Type : {type(date_dvf_example)}")
print(f"      Valeur : {date_dvf_example}")
print(f"      Timestamp : {pd.Timestamp(date_dvf_example)}")
print(f"      Normalize : {pd.Timestamp(date_dvf_example).normalize()}")

print(f"\n   Date macro :")
print(f"      Type : {type(date_macro_example)}")
print(f"      Valeur : {date_macro_example}")
print(f"      Timestamp : {pd.Timestamp(date_macro_example)}")
print(f"      Normalize : {pd.Timestamp(date_macro_example).normalize()}")

# Comparer directement
print(f"\n   Égalité stricte : {date_dvf_example == date_macro_example}")
print(f"   Égalité après normalize : {pd.Timestamp(date_dvf_example).normalize() == pd.Timestamp(date_macro_example).normalize()}")

print("\n" + "=" * 70)

🔍 DIAGNOSTIC DES DONNÉES TRIMESTRIELLES

📅 ÉTAPE 1 : Examiner les dates des séries macro
──────────────────────────────────────────────────────────────────────

🔹 TAUX CRÉDIT (df_taux_credit_moyen)
   Nombre de lignes : 26
   Type de Date_Trimestriel : datetime64[ns]

   Premières dates :
  Date_Trimestriel  Taux_moyen_credit_immo
0       2019-01-01                  0.0142
1       2019-02-01                  0.0129
2       2019-03-01                  0.0119
3       2019-04-01                  0.0113
4       2020-01-01                  0.0113
5       2020-02-01                  0.0129
6       2020-03-01                  0.0123
7       2020-04-01                  0.0120
8       2021-01-01                  0.0113
9       2021-02-01                  0.0106

   Dernières dates :
   Date_Trimestriel  Taux_moyen_credit_immo
16       2023-01-01                  0.0284
17       2023-02-01                  0.0329
18       2023-03-01                  0.0377
19       2023-04-01                  0.

/tmp/ipykernel_1809/2095794405.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_taux['Date_Trimestriel'] = pd.to_datetime(df_taux['Date_Trimestriel'], errors='coerce')
/tmp/ipykernel_1809/2095794405.py:43: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_pib['Date'] = pd.to_datetime(df_pib['Date'], errors='coerce')
